### This is the code for the web scraping functions using pygooglenews and other packages to scrape Google News according to a matrix of factors including author name, key words found in the titles of the peer-reviewed publications, DOI's and peer-reviewed journal names. 

### Import pygooglenews

In [15]:
import nltk
from nltk.corpus import stopwords
import re
import string
from collections import Counter

from pygooglenews import GoogleNews
import pandas as pd
import numpy as np

gn = GoogleNews() # now global mode? can specify country = US

In [1]:
import warnings
warnings.filterwarnings('ignore')

### Import .csv file 

In [16]:
def main(data):
    """Main function which accepts a CSV file as input. Invokes (i) all scraping functions, 
    (ii) function which consolidates a list of relevant keywords. Returns a csv file containing
    academic journal article title, author, and DOI appended to related news 
    articles and their corresponding URLS"""
#  df = pd.read_csv(data) # must be in same repository that the jupyter notebook is located in
df = pd.read_csv("UseCase2_Data.csv")

In [17]:
df
# To do:
# Value error checking for header names
# Write unit tests -- Ishika think of cases
# Write documentation for new code
# Update documentation for changes
# Linking frontend and backend stuff
# Repeat documention stuff for other use cases
# Update design docs
# Update package specs in environment
# Continuous integration (WTF LEARN THIS)
# Delete one use case from documentation and stick with only two use cases (Delaney doing 12/4/22)

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
0,J,"Pauker, K; Carpinella, C; Meyers, C; Young, DM...",NaN,NaN,NaN,"Pauker, Kristin; Carpinella, Colleen; Meyers, ...",NaN,NaN,The Role of Diversity Exposure in Whites' Redu...,SOCIAL PSYCHOLOGICAL AND PERSONALITY SCIENCE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000453466900007,View Full Record in Web of Science
1,J,"Cipollina, R; Pereira-Jorge, I; Sanchez, DT",NaN,NaN,NaN,"Cipollina, Rebecca; Pereira-Jorge, Izilda; San...",NaN,NaN,Perceptions of racial essentialism and social ...,GROUP PROCESSES & INTERGROUP RELATIONS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000871908600001,View Full Record in Web of Science
2,J,"Tullmann, K",NaN,NaN,NaN,"Tullmann, Katie",NaN,NaN,Visual essentialism & social kinds,JOURNAL FOR THE THEORY OF SOCIAL BEHAVIOUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000680713800001,View Full Record in Web of Science
3,J,"Mandalaywala, TM; Amodio, DM; Rhodes, M",NaN,NaN,NaN,"Mandalaywala, Tara M.; Amodio, David M.; Rhode...",NaN,NaN,Essentialism Promotes Racial Prejudice by Incr...,SOCIAL PSYCHOLOGICAL AND PERSONALITY SCIENCE,...,NaN,NaN,NaN,33163145.0,NaN,NaN,NaN,NaN,WOS:000439615800010,View Full Record in Web of Science
4,J,"James, D; Bonam, CM",NaN,NaN,NaN,"James, Drexler; Bonam, Courtney M.",NaN,NaN,Biogeographic ancestry information facilitates...,JOURNAL OF APPLIED SOCIAL PSYCHOLOGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000866440600001,View Full Record in Web of Science
5,J,"Small, PA; Major, B",NaN,NaN,NaN,"Small, Payton A.; Major, Brenda",NaN,NaN,Crossing the racial line: The fluidity vs. fix...,SELF AND IDENTITY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000484973800001,View Full Record in Web of Science
6,J,"Schudson, ZC; Gelman, SA",NaN,NaN,NaN,"Schudson, Zach C.; Gelman, Susan A.",NaN,NaN,Social constructionist and essentialist belief...,GROUP PROCESSES & INTERGROUP RELATIONS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000763838300001,View Full Record in Web of Science
7,J,"Levy, A; Nguyen, C; Slepian, ML; Gaither, S; P...",NaN,NaN,NaN,"Levy, Aharon; Nguyen, Christine; Slepian, Mich...",NaN,NaN,Categorizing a Face and Facing a Category: The...,PERSONALITY AND SOCIAL PSYCHOLOGY BULLETIN,...,NaN,NaN,NaN,35383507.0,NaN,NaN,NaN,NaN,WOS:000780213400001,View Full Record in Web of Science
8,J,"Yao, DJ; Chao, MM; Leung, AKY",NaN,NaN,NaN,"Yao, Donna J.; Chao, Melody M.; Leung, Angela ...",NaN,NaN,When Essentialism Facilitates Intergroup Confl...,JOURNAL OF CROSS-CULTURAL PSYCHOLOGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000463918700001,View Full Record in Web of Science
9,S,"Pauker, K; Tai, C; Ansari, S",NaN,"Rhodes, M",NaN,"Pauker, Kristin; Tai, Christine; Ansari, Shahana",NaN,NaN,Contextualizing the development of social esse...,DEVELOPMENT OF SOCIAL ESSENTIALISM,...,NaN,NaN,NaN,32564796.0,NaN,NaN,NaN,NaN,WOS:000609245800004,View Full Record in Web of Science


In [28]:
df = df.dropna(how ='all') # drop last n rows because google news functions 
#won't work with NaN or NA values, we need to add an ifelse/judgement to each function. We only can search for strings
df.dropna(how='all', axis=1, inplace=True)
df

author_cols = [col for col in df.columns if 'author' in col]+[col for col in df.columns if 'Author' in col]
#print(list(df.columns))
print(author_cols)
title_cols = [col for col in df.columns if 'title' in col]+[col for col in df.columns if 'Title' in col]
print(title_cols)
doi_labels = [col for col in df.columns if 'doi' in col]+[col for col in df.columns if 'DOI' in col]+[col for col in df.columns if 'Doi' in col]
print(doi_labels)


# write a function where we get indices of headers which contain the string "Author", "DOI" and "Article Title"

['Authors', 'Author Full Names']
['Article Title', 'Source Title', 'Book Series Title']
['DOI', 'DOI Link']


In [32]:
import difflib as difflib
difflib.get_close_matches('Author_Name', author_cols)
difflib.get_close_matches('Title', title_cols)

[]


### Use keywords in peer-reviewed article titles to reduce irrelevant return items from the scraping function

In [10]:
def key_words(df):
    article_titles = df['Article_Title'].to_list()

    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    punct = set(string.punctuation)
    
    for i in range(len(article_titles)):
        # lowercasing the text
        article_titles[i] = article_titles[i].lower()
        # remove stopwords
        article_titles[i] = " ".join([word for word in article_titles[i].split() if word not in stop_words])
        # remove unicode
        text = article_titles[i].encode(encoding='ascii', errors='ignore').decode()
        article_titles[i] = " ".join([word for word in text.split()])
        # remove market ticker and hashtag
        article_titles[i] = re.sub('\$', '', article_titles[i])
        article_titles[i] = re.sub('\#', '', article_titles[i])
        # remove punctuation
        article_titles[i] = "".join([ch for ch in article_titles[i] if ch not in punct])
        
    res = ' '.join(article_titles)
    res = res.split(' ')
    return Counter(res).most_common(10)

keyword_list = key_words(df)
keyword_list = (list(zip(*keyword_list))[0])
print(keyword_list)
#keywords(df)

KeyError: 'Article_Title'

### Function for scraping by author name only

In [ ]:
# search = gn.search(‘author name’) #search by author name
def scraping_author(df):
    """This function scrapes google news for anything matching the 
    scholarly author name in df"""
    #df_author = df.dropna([df.iloc'Author_Name'])
    df_author = df[df.Author_Name.notnull()]

    stories = []
    for i in range(len(df_author)):
        search = gn.search(df_author.iloc[i].loc['Author_Name'])
        newsitem = search['entries']
        for item in newsitem:
            story = {
                'Author_Name': df_author.iloc[i].loc['Author_Name'],
                'Article_Title': df_author.iloc[i].loc['Article_Title'],
                'Article_DOI': df_author.iloc[i].loc['Article_DOI'],
                'news_title': item.title, #change to 'news_title' for clarity
                'news_link': item.link
            }
            
            stories.append(story)    
    return stories
scraping_author_df = pd.DataFrame(scraping_author(df))# this puts the output in a long form dataframe/
#scraping_author_df 
key_count = [0]*len(scraping_author_df)
for i in range(len(scraping_author_df)):
    for j in range(len(keyword_list)):
        #print(keyword_list[j])
        if keyword_list[j] in scraping_author_df.iloc[i].loc['news_title']:
            key_count[i] = key_count[i]+1

scraping_author_df['key_count'] = key_count
        

In [ ]:
print(scraping_author_df)

### Function for scraping by DOI

In [90]:
# search = gn.search(‘Article_DOI’) #search by reference to article DOI

def scraping_doi(df):
    '''This function scrapes google news for anything matching the 
    scholarly DOI in df'''
    stories = []
    df_doi = df[df.Article_DOI.notnull()]
    for i in range(len(df_doi)):
        search = gn.search(df_doi.iloc[i].loc['Article_DOI'])
        newsitem = search['entries']
        for item in newsitem:
            story = {
                'Author_Name': df_doi.iloc[i].loc['Author_Name'],
                'Article_Title': df_doi.iloc[i].loc['Article_Title'],
                'Article_DOI': df_doi.iloc[i].loc['Article_DOI'],
                'news_title': item.title, #change to 'news_title' for clarity
                'news_link': item.link
            }
            
            stories.append(story)    
    return stories
scraping_doi_df = pd.DataFrame(scraping_doi(df)) # this puts the output in a long form dataframe

scraping_doi_df 

,Author_Name,Article_Title,Article_DOI,news_title,news_link
0,Heather Sykes,Transsexual and Transgender Policies in Sport,10.1123/wspaj.15.1.3,“Debate” about Trans Girls and Women in School...,https://news.google.com/__i/rss/rd/articles/CB...
1,Emma Hilton,Transgender women in the female category of sp...,10.1007/s40279-020-01389-3,FINA introduces a new policy for transgender s...,https://news.google.com/__i/rss/rd/articles/CB...
2,Timothy Roberts,Effect of gender affirming hormones on athleti...,10.1136/bjsports-2020-102329,Effect of gender affirming hormones on athleti...,https://news.google.com/__i/rss/rd/articles/CB...
3,Taryon Knox,Transwomen in elite sport: scientific and ethi...,10.1136/medethics-2018-105208,Gender binary in elite sports should be abando...,https://news.google.com/__i/rss/rd/articles/CB...


### Function for scraping by exact article title 

In [91]:
# search = gn.search(‘Article_Title’) #search by reference to peer reviewed article title
def scraping_title(df):
    stories = []
    df_article_title = df[df.Article_Title.notnull()]
    for i in range(len(df_article_title)):
        search = gn.search(df_article_title.iloc[i].loc['Article_Title'])
        newsitem = search['entries']
        for item in newsitem:
            story = {
                'Author_Name': df_article_title.iloc[i].loc['Author_Name'],
                'Article_Title': df_article_title.iloc[i].loc['Article_Title'],
                'Article_DOI': df_article_title.iloc[i].loc['Article_DOI'],
                'news_title': item.title, #change to 'news_title' for clarity
                'news_link': item.link
            }
            
            stories.append(story)    
    return stories

scraping_title_df = pd.DataFrame(scraping_title(df)) # this puts the output in a long form dataframe

scraping_title_df

,Author_Name,Article_Title,Article_DOI,news_title,news_link
0,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,Transgender Adults Have Higher Rates Of Disabi...,https://news.google.com/__i/rss/rd/articles/CB...
1,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,Transgender Legal Battles: A Timeline - JSTOR ...,https://news.google.com/__i/rss/rd/articles/CB...
2,Jonathan Ospina-Betancurt,The End of Compulsory Gender Verification: Is ...,10.1007/s10508-021-02073-x,“They're Chasing Us Away from Sport”: Human Ri...,https://news.google.com/__i/rss/rd/articles/CB...
3,Katrina Karkazis,The misuses of “biological sex”,10.1016/S0140-6736(19)32764-3,Biology Lecturer’s Comments on Biological Sex ...,https://news.google.com/__i/rss/rd/articles/CB...
4,Vanessa Heggie,"Testing sex and gender in sports; reinventing,...",10.1016/j.endeavour.2010.09.005,Testosterone and Athletics: The Biology of the...,https://news.google.com/__i/rss/rd/articles/CB...
...,...,...,...,...,...
303,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,Nancy Hogshead-Makar Explains Problems With Li...,https://news.google.com/__i/rss/rd/articles/CB...
304,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,The death of women’s sports? - The Citizen.com,https://news.google.com/__i/rss/rd/articles/CB...
305,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,"Without NCAA Action, Effects of Lia Thomas Sit...",https://news.google.com/__i/rss/rd/articles/CB...
306,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,Young transgender athletes caught in middle of...,https://news.google.com/__i/rss/rd/articles/CB...


In [92]:
scrape_out = scraping_author_df.append([scraping_doi_df, scraping_title_df]) #append all df's together
scrape_out

,Author_Name,Article_Title,Article_DOI,news_title,news_link,key_count
0,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,Johnson Elementary girls win county championsh...,https://news.google.com/__i/rss/rd/articles/CB...,0.0
1,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,Transgender Legal Battles: A Timeline - JSTOR ...,https://news.google.com/__i/rss/rd/articles/CB...,1.0
2,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,Thorpe questions FINA's trans swimming ban: Th...,https://news.google.com/__i/rss/rd/articles/CB...,0.0
3,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,Real Estate Transactions for Nov. 24 - Zip06.com,https://news.google.com/__i/rss/rd/articles/CB...,0.0
4,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203,"St. Tammany property transfers, Oct.18-24, 202...",https://news.google.com/__i/rss/rd/articles/CB...,0.0
...,...,...,...,...,...,...
303,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,Nancy Hogshead-Makar Explains Problems With Li...,https://news.google.com/__i/rss/rd/articles/CB...,NaN
304,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,The death of women’s sports? - The Citizen.com,https://news.google.com/__i/rss/rd/articles/CB...,NaN
305,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,"Without NCAA Action, Effects of Lia Thomas Sit...",https://news.google.com/__i/rss/rd/articles/CB...,NaN
306,John Devine,"Gender, Steroids, and Fairness in Sport",10.1080/17511321.2017.1404627,Young transgender athletes caught in middle of...,https://news.google.com/__i/rss/rd/articles/CB...,NaN


In [93]:
scrape_out.size
scrape_out.tail(1)
scrape_out = scrape_out[scrape_out.key_count != 0]

scrape_out.head()
scrape_out.tail()
print(scrape_out)

          Author_Name                                      Article_Title  \
1    Laurel Westbrook  Doing Gender, Determining Gender: Transgender ...   
7    Laurel Westbrook  Doing Gender, Determining Gender: Transgender ...   
14   Laurel Westbrook  Doing Gender, Determining Gender: Transgender ...   
17   Katrina Karkazis                    The misuses of “biological sex”   
19   Katrina Karkazis                    The misuses of “biological sex”   
..                ...                                                ...   
303       John Devine            Gender, Steroids, and Fairness in Sport   
304       John Devine            Gender, Steroids, and Fairness in Sport   
305       John Devine            Gender, Steroids, and Fairness in Sport   
306       John Devine            Gender, Steroids, and Fairness in Sport   
307       John Devine            Gender, Steroids, and Fairness in Sport   

                       Article_DOI  \
1         10.1177/0891243213503203   
7         1

In [94]:
scrape_out.drop_duplicates(subset=['news_link'])
scrape_out.head()
scrape_out.tail()
print(scrape_out)

          Author_Name                                      Article_Title  \
1    Laurel Westbrook  Doing Gender, Determining Gender: Transgender ...   
7    Laurel Westbrook  Doing Gender, Determining Gender: Transgender ...   
14   Laurel Westbrook  Doing Gender, Determining Gender: Transgender ...   
17   Katrina Karkazis                    The misuses of “biological sex”   
19   Katrina Karkazis                    The misuses of “biological sex”   
..                ...                                                ...   
303       John Devine            Gender, Steroids, and Fairness in Sport   
304       John Devine            Gender, Steroids, and Fairness in Sport   
305       John Devine            Gender, Steroids, and Fairness in Sport   
306       John Devine            Gender, Steroids, and Fairness in Sport   
307       John Devine            Gender, Steroids, and Fairness in Sport   

                       Article_DOI  \
1         10.1177/0891243213503203   
7         1